In [ ]:
import os
import yt_dlp
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed
import yaml

# 로깅 설정
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)


def sanitize_filename(title):
    return "".join(
        [c for c in title if c.isalpha() or c.isdigit() or c == " "]
    ).rstrip()


def download_audio(video_info, download_path):
    try:
        if isinstance(video_info, str):  # URL 문자열인 경우
            url = video_info
            ydl_opts = {"extract_info": True}
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(url, download=False)
                title = info["title"]
        else:  # video_info 딕셔너리인 경우
            url = video_info["url"]
            title = video_info["title"]

        safe_title = sanitize_filename(title)
        filename = os.path.join(download_path, f"{safe_title}")

        ydl_opts = {
            "format": "bestaudio/best",
            "postprocessors": [
                {
                    "key": "FFmpegExtractAudio",
                    "preferredcodec": "mp3",
                    "preferredquality": "192",
                }
            ],
            "outtmpl": filename,
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        logging.info(f"Downloaded: {filename}")
    except Exception as e:
        logging.error(f"Error downloading {url}: {str(e)}")


def process_url(url, download_path):
    if "playlist" in url:
        process_playlist(url, download_path)
    else:
        download_audio(url, download_path)


def process_playlist(playlist_url, download_path):
    ydl_opts = {
        "extract_flat": "in_playlist",
        "dump_single_json": True,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            result = ydl.extract_info(playlist_url, download=False)

        if "entries" in result:
            video_list = result["entries"]

            with ThreadPoolExecutor(max_workers=5) as executor:
                future_to_url = {
                    executor.submit(download_audio, video, download_path): video
                    for video in video_list
                }
                for future in as_completed(future_to_url):
                    video = future_to_url[future]
                    try:
                        future.result()
                    except Exception as e:
                        logging.error(f"Error processing {video['url']}: {str(e)}")
        else:
            logging.warning("No videos found in the playlist.")
    except Exception as e:
        logging.error(f"Error processing playlist: {str(e)}")


def load_yaml(path):
    with open(path, "r", encoding="utf-8") as file:
        return yaml.safe_load(file)


config = load_yaml("../config/metadata_tagger.yaml")
url_type = config["settings"]["url_type"]


if url_type not in ["playlist", "video"]:
    logging.error("잘못된 선택입니다. 'playlist' 또는 'video'를 입력해주세요.")
    exit()

url = config["settings"]["url"]
download_path = config["settings"]["download_path"]

logging.info(f"파일들이 다음 경로에 저장됩니다: {download_path}")
process_url(url, download_path)